In [1]:
import numpy as np
import subprocess
import time
import copy
from utils.opt_utils import *
import os
import h5py
import pickle
from qutip import *
import utils.pulse_configs as pc

In [2]:
config_dict = pc.config_dict

In [3]:
# Defining relevant values
wq = config_dict["qubit"]["selective_freqs"]["g"]["ge"]
wa_B = config_dict["storage_B"]["freq"]

In [4]:
# ========== OPTIONS ========== #
max_segs = 20
time_start = 0.0000000
time_stop = 0.000001
init_amp = 1500
n_steps = 501

num_drives = 2
chs = [2, 4]
init_freqs = [wq, wa_B]
verbose = True
plot_opt_pulses = False
plot_pulse_viewer = False

save_dir = 'save_data'
hdf5_name = os.path.join(save_dir, time.strftime('%Y%m%d-%H%M%S.h5'))
epochs = 100
train_batch_size = 20
qubit_amp_scale = 4
cavity_amp_scale = 4
freq_scale = 0.001
# ========== OPTIONS ========== #

drive_freqs = np.array(init_freqs)

qscale = []
cscale = []
for i in range(2):
    qscale.append(init_amp * qubit_amp_scale)
    cscale.append(init_amp * cavity_amp_scale)
    # cscale.append(init_amp * cavity_amp_scale)

t_segs, amp_segs = setup_segs(2 * num_drives, time_start, time_stop, init_amp)

# Create blank history arrays for storing optimal / past values
time_hist = []
amp_hist = []
freq_hist = []
cost_hist = []

In [7]:
# Run vqe, etc
hdf5_start_index = 4
start_segs = 2
for i in range(max_segs):
    
    temp_amp_scale = copy.deepcopy(amp_segs)
    if np.shape(temp_amp_scale)[0] < 2:
        temp_amp_scale[:, :] = np.vectorize(max)(np.abs(amp_segs * qubit_amp_scale), init_amp * np.ones(np.shape(amp_segs)) / qubit_amp_scale)
    else:
        temp_amp_scale[:2, :] = np.vectorize(max)(np.abs(amp_segs[:2] * qubit_amp_scale), init_amp * np.ones(np.shape(amp_segs[:2])) / qubit_amp_scale)
        temp_amp_scale[2:, :] = np.vectorize(max)(np.abs(amp_segs[2:] * cavity_amp_scale), init_amp * np.ones(np.shape(amp_segs[2:])) / cavity_amp_scale)
    temp_freq_scale = list(freq_scale * np.array(drive_freqs))

    client_args = [t_segs, chs, verbose, plot_opt_pulses, plot_pulse_viewer]
    server_args = [epochs, train_batch_size, amp_segs, drive_freqs, temp_amp_scale, temp_freq_scale, hdf5_name]

    
    # Save args for rl client
    cf_name = "temp_files/client_args.txt"
    with open(cf_name, "wb") as fp:
        pickle.dump(client_args, fp)
    fp.close()
    
    # Save args for rl server
    sf_name = "temp_files/server_args.txt"
    with open(sf_name, "wb") as fp:
        pickle.dump(server_args, fp)
    fp.close()
    

    os.system('cmd /c python ./exp_ctrl_vqe_run_rl_scripts.py')
    
    # Give time for files to be updated etc
    time.sleep(1)
    
    opt_amps = []
    opt_freqs = []
    with h5py.File(hdf5_name, "r") as f:
        opt_res_index = np.argmax(f[str(i + hdf5_start_index)]["evaluation"]["rewards"][()])
        opt_result = f[str(i + hdf5_start_index)]["evaluation"]["rewards"][()][opt_res_index]
        for j in range(2 * num_drives):
            opt_amps.append([f[str(i + hdf5_start_index)]["evaluation"]["actions"][f'pulse_array_{j}'][()][opt_res_index]])
        for j in range(num_drives):
            opt_freqs.append(f[str(i + hdf5_start_index)]["evaluation"]["actions"][f'freq_{j}'][()][opt_res_index])
    
    # updates amplitudes and frequencies with optimized values and reshape
    amp_segs = np.array(opt_amps)
    amp_segs = np.reshape(amp_segs, (num_drives * 2, int(len(amp_segs.flatten()) / (num_drives * 2))))
    
    print(f'================')
    print(f'num segs: {i + start_segs} ')
    print(f'opt_amps: {amp_segs}')
    print(f'opt_freqs: {opt_freqs}')
    print(f'opt_result: {opt_result}')

    # save values to history arrays
    time_hist.append(t_segs)
    amp_hist.append(amp_segs)
    freq_hist.append(drive_freqs)
    cost_hist.append(opt_result)

    for i in range(2):
        qscale.append(init_amp * qubit_amp_scale)
        cscale.append(init_amp * cavity_amp_scale)
        # cscale.append(init_amp * cavity_amp_scale)

    # split segments and return to start of loop
    if (i < max_segs - 1):
        t_segs, amp_segs = split_segs(t_segs, amp_segs)

num segs: 2 
opt_amps: [[2037.4072  2278.032  ]
 [1978.5233  2627.8347 ]
 [ 651.10266  533.9048 ]
 [1661.9077  1006.0288 ]]
opt_freqs: [5322.494, 6090.774]
opt_result: [0.2556413]
num segs: 3 
opt_amps: [[1861.0522  1163.9426  3298.7708 ]
 [2061.7737  1267.5237  3116.5066 ]
 [ 687.0707   625.60565  726.68536]
 [1346.8394  1175.073    987.48505]]
opt_freqs: [5322.734, 6091.6426]
opt_result: [0.40267968]
num segs: 4 
opt_amps: [[1261.4546  2405.1694   762.82227 4862.365  ]
 [2642.2556  1706.1279  1335.7029  3214.0393 ]
 [ 836.9945   495.14984  538.1565  1191.7106 ]
 [1051.5334   827.0946  1412.3511  1330.4308 ]]
opt_freqs: [5322.636, 6090.4536]
opt_result: [0.35886052]
num segs: 5 
opt_amps: [[1748.7461  1831.4419  3322.1438   867.4953  7390.714  ]
 [2064.7761  4023.891   1303.1815  1490.7366  2873.9126 ]
 [ 818.4445   441.2496   888.9213  1308.6202   834.1752 ]
 [ 848.9824   920.07367 1237.5476  2014.0078  1336.8407 ]]
opt_freqs: [5323.0454, 6091.9927]
opt_result: [0.39136207]
num segs:

KeyError: "Unable to open object (object 'evaluation' doesn't exist)"